In [1]:
import pandas as pd
import numpy as np
import datetime

In [9]:
inflation=pd.read_csv("PH_inflation_historical.csv")
rates=pd.read_csv("PHreferencerates.csv")

# convert DATE to datetime format
# only have inflation 2013 onwards (since base=2012)
# get only rates 2013 onwards as well
rates['DATE']=pd.to_datetime(rates['DATE'])
rates=rates[rates.DATE>'2013-01-01']

# add DATE column to inflation
inflation['DATE']=inflation.apply(lambda row: datetime.datetime(int(row.Year),int(row.Month),1),axis=1)

In [11]:
inflation.tail()

,Year,Month,Inflation,DATE
87,2020,4,2.2,2020-04-01
88,2020,5,2.1,2020-05-01
89,2020,6,2.5,2020-06-01
90,2020,7,2.7,2020-07-01
91,2020,7,2.7,2020-07-01


In [12]:
combined=pd.merge_asof(rates,inflation,on='DATE')

In [13]:
# create the real yields dataframe
tenors=['1M','3M','6M','1Y','2Y','3Y','4Y','5Y','7Y','10Y','20Y']
realyields=pd.DataFrame([combined[x]-combined['Inflation'] for x in tenors]).transpose()
realyields.columns=tenors
realyields['DATE']=combined['DATE']
realyields.set_index('DATE',inplace=True)
realyields.head()

,1M,3M,6M,1Y,2Y,3Y,4Y,5Y,7Y,10Y,20Y
DATE,,,,,,,,,,,
2013-01-02,-2.1,-2.5050,-2.225,-2.0000,-0.0785,0.4500,0.8250,0.9000,1.1000,1.400,2.840
2013-01-03,-2.3,-2.5050,-2.225,-2.0750,-0.4707,0.4250,0.7941,0.8850,1.1000,1.385,2.835
2013-01-04,-2.3,-2.5100,-2.230,-2.2000,-0.4899,0.4000,0.8003,0.8800,1.1250,1.370,2.785
2013-01-07,-2.3,-2.6250,-2.350,-2.0850,-0.5000,0.4000,0.7429,0.8012,1.1250,1.365,2.770
2013-01-08,-2.3,-2.7538,-2.525,-2.0858,-0.6245,0.3437,0.7472,0.8000,1.1745,1.350,2.765


In [14]:
realyields.tail()

,1M,3M,6M,1Y,2Y,3Y,4Y,5Y,7Y,10Y,20Y
DATE,,,,,,,,,,,
2020-08-19,-1.588,-1.497,-1.250,-0.914,-0.654,-0.526,-0.413,-0.308,-0.137,-0.059,0.831
2020-08-20,-1.589,-1.509,-1.263,-0.913,-0.688,-0.550,-0.427,-0.311,-0.129,-0.061,0.878
2020-08-24,-1.582,-1.512,-1.257,-0.913,-0.665,-0.526,-0.412,-0.308,-0.121,-0.011,0.887
2020-08-25,-1.581,-1.509,-1.269,-0.915,-0.664,-0.540,-0.427,-0.320,-0.139,-0.016,0.914
2020-08-26,-1.580,-1.506,-1.271,-0.910,-0.628,-0.476,-0.336,-0.210,-0.041,0.085,0.784


In [15]:
realyields.to_csv("PHrealyields.csv",index=False)

## Get 1 year and 5 year Z scores of real yields

### 1-year Z-score

In [16]:
#Get the rolling average real yield
realyield_ave_1year=realyields.rolling(260).mean()[259:]
realyield_ave_1year.head()

,1M,3M,6M,1Y,2Y,3Y,4Y,5Y,7Y,10Y,20Y
DATE,,,,,,,,,,,
2014-01-27,-2.014543,-2.031230,-1.866242,-1.598715,-0.402375,-0.277254,0.272132,0.268503,0.675919,0.831705,1.556597
2014-01-28,-2.015024,-2.031210,-1.863632,-1.597080,-0.406847,-0.283405,0.266521,0.264830,0.672650,0.827857,1.551155
2014-01-29,-2.014736,-2.031191,-1.861612,-1.595638,-0.411026,-0.289143,0.262678,0.260850,0.669442,0.824134,1.544674
2014-01-30,-2.014351,-2.031153,-1.859670,-1.593715,-0.413084,-0.294240,0.258812,0.255896,0.666197,0.820491,1.539443
2014-02-03,-2.013005,-2.028749,-1.856785,-1.591369,-0.415392,-0.298759,0.255097,0.252815,0.663826,0.817197,1.534655


In [17]:
#Get the rolling std of real yields
realyield_vol_1year=realyields.rolling(260).std()[259:]
realyield_vol_1year.head()

,1M,3M,6M,1Y,2Y,3Y,4Y,5Y,7Y,10Y,20Y
DATE,,,,,,,,,,,
2014-01-27,0.874291,0.883637,0.823627,0.769504,0.606144,0.588012,0.542910,0.483755,0.503960,0.513274,0.535223
2014-01-28,0.874373,0.883627,0.823562,0.769099,0.608033,0.588739,0.544710,0.482566,0.503955,0.512747,0.529284
2014-01-29,0.874291,0.883617,0.823316,0.768551,0.612193,0.589079,0.544498,0.481698,0.503879,0.512200,0.523791
2014-01-30,0.874187,0.883596,0.823039,0.767658,0.613357,0.588825,0.544234,0.481929,0.503650,0.511671,0.518117
2014-02-03,0.874014,0.882824,0.822628,0.767083,0.614812,0.587985,0.544214,0.480989,0.502932,0.510908,0.512422


In [18]:
# Get same size
realyields_1year=realyields[259:]
# Get difference (X-mu)
diff_realyield_1year=realyields_1year.subtract(realyield_ave_1year)
# Divide by vol to get Z
realyield_Z_1year=diff_realyield_1year.divide(realyield_vol_1year)
realyield_Z_1year=realyield_Z_1year.round(decimals=2)
realyield_Z_1year.head()

,1M,3M,6M,1Y,2Y,3Y,4Y,5Y,7Y,10Y,20Y
DATE,,,,,,,,,,,
2014-01-27,-0.21,0.53,0.81,-0.13,-1.91,-1.42,-0.64,-1.47,-0.85,-0.88,-0.38
2014-01-28,-0.24,-0.53,0.39,0.03,-1.37,-1.47,-1.65,-0.66,-0.84,-0.84,-0.24
2014-01-29,-0.24,-0.53,0.20,-0.14,-1.87,-1.32,-0.86,-0.85,-0.80,-0.80,-0.75
2014-01-30,-0.21,-0.53,0.16,-0.14,-1.00,-1.07,-0.85,-1.38,-0.76,-0.78,-0.22
2014-02-03,0.07,0.03,0.31,0.15,-1.11,-0.81,-0.88,-0.53,-0.31,-0.60,-0.02


### 5-year Z score

In [19]:
#Get the rolling average real yield
realyield_ave_5year=realyields.rolling(260*5).mean()[260*5-1:]
#Get the rolling std of real yields
realyield_vol_5year=realyields.rolling(260*5).std()[260*5-1:]
# Get same size
realyields_5year=realyields[260*5-1:]
# Get difference (X-mu)
diff_realyield_5year=realyields_5year.subtract(realyield_ave_5year)
# Divide by vol to get Z
realyield_Z_5year=diff_realyield_5year.divide(realyield_vol_5year)
realyield_Z_5year=realyield_Z_5year.round(decimals=2)
realyield_Z_5year.head()

,1M,3M,6M,1Y,2Y,3Y,4Y,5Y,7Y,10Y,20Y
DATE,,,,,,,,,,,
2018-05-16,-0.62,-0.34,-0.24,-0.21,-0.51,-0.54,-0.19,-0.36,-0.39,-0.00,-0.04
2018-05-17,-0.59,-0.34,-0.22,-0.21,-0.60,-0.55,-0.45,-0.34,-0.38,0.08,-0.00
2018-05-18,-0.53,-0.33,-0.21,-0.21,-0.56,-0.55,-0.12,-0.25,-0.36,0.15,0.00
2018-05-21,-0.57,-0.33,-0.22,-0.22,-0.61,-0.55,-0.46,-0.29,-0.38,0.25,0.03
2018-05-22,-0.54,-0.35,-0.24,-0.20,-0.62,-0.54,-0.08,-0.31,-0.38,-0.18,0.11


In [20]:
latest1yr_Z_realyield=realyield_Z_1year.tail(1).transpose()
latest1yr_Z_realyield.columns=['Real Yield 1-year Z']
latest5yr_Z_realyield=realyield_Z_5year.tail(1).transpose()
latest5yr_Z_realyield.columns=['Real Yield 5-year Z']

In [21]:
latestZ_realyields=pd.concat([latest1yr_Z_realyield,latest5yr_Z_realyield],axis=1)
latestZ_realyields

,Real Yield 1-year Z,Real Yield 5-year Z
1M,-1.92,-1.31
3M,-1.92,-1.27
6M,-1.84,-1.41
1Y,-1.74,-1.39
2Y,-1.62,-1.68
3Y,-1.58,-1.71
4Y,-1.54,-1.95
5Y,-1.51,-1.95
7Y,-1.51,-2.04
10Y,-1.52,-2.16


In [22]:
latestZ_realyields.to_csv("latestZ_realyields.csv")

In [70]:
import seaborn as sns

In [164]:
#Add colors to easily see which are attractive and which arent
#source: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
def color_font(val):
    if val < 0:
        color='red'
    elif val<1.65:
        color='orange'
    else:
        color='green'
    return 'color: %s' %color

In [165]:
latestZ_realyields.style.applymap(color_font)

,Real Yield 1-year Z,Real Yield 5-year Z
1M,-2.010000,-0.900000
3M,-2.070000,-0.890000
6M,-2.120000,-1.140000
1Y,-2.170000,-1.260000
2Y,-2.060000,-1.620000
3Y,-2.010000,-1.670000
4Y,-1.970000,-1.970000
5Y,-1.940000,-1.990000
7Y,-1.920000,-2.080000
10Y,-1.920000,-2.200000
